# Carga de datos

In [1]:
import pandas as pd
import numpy as np
import gzip, pickle, pickletools
import time

In [2]:
basepath = "https://www.dropbox.com/s/"

In [3]:
# Archivo de prestamos
df_prestamos = pd.read_json(basepath + "q38zr341seq7rkf/joinTablas.json?dl=1")

In [5]:
df_prestamos.columns

Index(['RowID', 'Fecha', 'IDItem', 'NumeroUbicacion', 'Dewey', 'Ubicacion',
       'Llave', 'Programa', 'Facultad', 'IDUsuario', 'Year', 'Signatura',
       'TipoItem', 'FechaCreacion', 'Autor', 'Titulo', 'AnioPublicacion',
       'DeweyEspecifico', 'TemaDewey', 'DeweyUnidad', 'DeweyDecena',
       'DeweyCentena', 'Temas', 'Union'],
      dtype='object')

In [11]:
df_prestamos['Temas']

0                     [arquitectur, histori, amer, latin]
1               [arquitectur, colonial, histori, colombi]
2                                                [quimic]
3                                                [algebr]
4                                   [arquitectur, paisaj]
                               ...                       
489732                              [filosofi, educacion]
489733                              [filosofi, educacion]
489734                                    [arte, histori]
489735    [exam, admision, gradu, administracion, empres]
489736                                  [cuent, austriac]
Name: Temas, Length: 489474, dtype: object

In [ ]:
df_temas = pd.read_json(basepath + 'a2wops7yxdjd2ea/Libros.json?dl=1')

In [ ]:
df_pesos =  pd.read_json(basepath + 've3qvkt0s6aymzp/pesos_dewey_unidad_difusa.json?dl=1')

In [ ]:
df_pesos.rename(columns={'Llave':'Llaves', 'DeweyUnidad': 'cluster', 'Fecha': 'timestamp', 'pesos': 'Peso del prestamos'}, inplace=True)

In [ ]:
df_pesos.loc[df_pesos['Llaves'] =='#XL_EVAL_ERROR#', 'Llaves'] = 0

In [ ]:
df_pesos

In [ ]:
df_pesos_full =  pd.read_json(basepath + 'zuwne8aii9dknvs/pesos_clustering_unidad.json?dl=1')

# Funciones auxiliares

In [ ]:
#Para obtener el nombre de un material a partir del título
def itemIdToMaterial(id):
    return df_prestamos.loc[df_prestamos['Llave'] == id]['Titulo'].unique()

In [ ]:

def itemIdToDewey(id):
    item = (df_prestamos.loc[df_prestamos['Llave'] == id]['Dewey'].unique())
    if len(item) == 0:
        return  (df_prestamos.loc[df_prestamos['Llave'] == str(id)]['Dewey'].unique())[0]
    else:
        return item[0]

In [ ]:
def itemIdToThemes(id):
    tematicas = df_temas.loc[df_temas['Llaves'] == str(id)]['Tema 670'].values
    if len(tematicas) == 0:
        return []
    else:
        return tematicas

In [ ]:

def generateClusterInfo(cluster_eval, df_eval):
    dfP = df_eval.copy()
    user_p = dfP.loc[dfP['cluster'] == cluster_eval]
    tematicas =[]
    for llave in user_p['Llaves'].unique():
        tematicas.append(itemIdToThemes((llave)))

    user_p['Tematicas'] = tematicas
    display(user_p)
    deweys = ((dfP.loc[dfP['cluster'] == cluster_eval]['Dewey'].unique()))
    print((dfP.loc[dfP['cluster'] == cluster_eval]['Dewey'].unique()))

    tematicas = []
    for tems in user_p['Tematicas']:
        for tema in tems:
            for temasub in tema:
                if temasub in tematicas:
                    continue
                else:
                    tematicas.append(temasub)

    return user_p, deweys, set(tematicas)

In [ ]:
def generateRecsInfopreds(preds):
    recs = preds
    data_recs_asr = []
    for ids in recs:
        data_recs_asr.append({'Titulo':itemIdToMaterial(ids), 
                         #'Rating':ratings[1], 
                          'Dewey':itemIdToDewey(ids), 
                         'Tematicas':itemIdToThemes(ids),
                          'Llave': ids})

    df_recs_asr = pd.DataFrame(data_recs_asr)
    display(df_recs_asr)
    deweys = print((df_recs_asr['Dewey'].unique()))
    print((df_recs_asr['Dewey'].unique()))

    tematicas = []
    for tems in df_recs_asr['Tematicas']:
        for tema in tems:
            for temasub in tema:
                if temasub in tematicas:
                    continue
                else:
                    tematicas.append(temasub)

    return df_recs_asr, deweys, set(tematicas)

In [ ]:
import sys
# sys.path.append(basePath + 'UAQUE/Recommendations')
# sys.path.append(basePath + 'UAQUE')
sys.path.append('./')

In [ ]:
import math
import numpy as np
    
def generate_recommendations_for_user(id_user, predictor):
    pertenencias = df_pesos_full.loc[df_pesos_full["IDUsuario"] == id_user]
    grupos = pertenencias["Cluster"].unique()
    preds = {}
    for gr in grupos:
        preds[gr] = predictor.predict_for_cluster(cluster=gr).values
        
    final_preds = []
    for key, value in preds.items():
        pert = pertenencias.loc[pertenencias["Cluster"] == key]["Pertenencia"].values[0]
        if pert < 0.1:
            pert = 0.1
        else:
            pert = round(pert,1)

        many_to_take = int(10 * pert)
        recs_to_add = np.random.choice(preds[key], many_to_take)
        final_preds.extend(recs_to_add)
        
    return final_preds

In [19]:
pertenencias = df_pesos_full.loc[df_pesos_full["IDUsuario"] == "6466dbb15c41fdacb59eb1179817958de2c57191"]
grupos = pertenencias["Cluster"].unique()

In [20]:
grupos

array([813, 823, 621, 833])

# SAR

In [17]:
from sar_recommeder import SarRecommeder

/opt/anaconda3/envs/reco_base/lib/python3.7/site-packages/papermill/iorw.py:50: FutureWarning: pyarrow.HadoopFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.HadoopFileSystem instead.
  from pyarrow import HadoopFileSystem


In [18]:
model_path = "sar_predictor150k_10211610.pkl"

In [19]:
with gzip.open(model_path, 'rb') as f:
    p = pickle.Unpickler(f)
    sar = p.load()
sar_predictor = sar


In [21]:
import gzip, pickle, pickletools
filepath = "sar_trained_model_du.pkl"
with gzip.open(filepath, "wb") as f:
    pickled = pickle.dumps(sar_predictor, protocol=4)
    optimized_pickle = pickletools.optimize(pickled)
    f.write(optimized_pickle)

In [37]:
!pip3 install blosc

     |████████████████████████████████| 3.2 MB 3.2 MB/s eta 0:00:01


In [40]:
import _pickle as pickle

In [30]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [73]:
sar_predictor_temas = SarRecommeder(df_pesos, n=df_pesos.shape[0])

In [ ]:
# Entrenando modelo
sar_predictor_temas.fit()

2021-10-22 22:29:03,041 INFO     Collecting user affinity matrix
2021-10-22 22:29:03,045 INFO     Calculating time-decayed affinities
2021-10-22 22:29:03,071 INFO     Creating index columns
2021-10-22 22:29:03,153 INFO     Calculating normalization factors
2021-10-22 22:29:03,179 INFO     Building user affinity sparse matrix
2021-10-22 22:29:03,183 INFO     Calculating item co-occurrence
2021-10-22 22:29:04,309 INFO     Calculating item similarity
2021-10-22 22:29:04,310 INFO     Using jaccard based similarity


In [ ]:
# Obteniendo Predicciones
top_k_temas = sar_predictor_temas.predict()

In [22]:
sar_predictor_temas.evaluate_model()

Model:	
Top K:	10
MAP:	0.001990
NDCG:	0.023119
Precision@K:	0.021002
Recall@K:	0.005584
RMSE:	3.704958
MAE:	3.687443
R2:	-5227.756267
Exp var:	-48.720965


In [ ]:
final_preds = generate_recommendations_for_user(id_user="6466dbb15c41fdacb59eb1179817958de2c57191", predictor=sar_predictor)

In [52]:
# Info Recomendaciones
generateRecsInfopreds(final_preds)

,Titulo,Dewey,Tematicas,Llave
0,[Adiós a las armas Ernest Hemingway ; traducci...,810,"[[novela, estadounidense]]",906848
1,[La torre oscura II la llegada de los tres Ste...,810,"[[novela, estadounidense]]",824586
2,[El horror de Dunwich Howard Phillips Lovecraf...,810,"[[literatura, estadounidense]]",860720
3,[La habitación cerrada y otros cuentos de terr...,810,"[[cuento, estadounidense]]",551719
4,[La torre oscura II la llegada de los tres Ste...,810,"[[novela, estadounidense]]",824586
5,[Patriot games Thomas Leo Clancy],810,"[[novela, estadounidense]]",850455
6,[La torre oscura II la llegada de los tres Ste...,810,"[[novela, estadounidense]]",824586
7,[The cleft Doris May Lessing],820,[],747541
8,[Un espía perfecto John Le Carré ; traducción ...,820,"[[novela, inglés]]",710677
9,[Digital communications fundamentals and appli...,620,"[[comunicación, digital]]",709655


['810' '820' '620' '830']
['810' '820' '620' '830']


(                                               Titulo Dewey  \
 0   [Adiós a las armas Ernest Hemingway ; traducci...   810   
 1   [La torre oscura II la llegada de los tres Ste...   810   
 2   [El horror de Dunwich Howard Phillips Lovecraf...   810   
 3   [La habitación cerrada y otros cuentos de terr...   810   
 4   [La torre oscura II la llegada de los tres Ste...   810   
 5                   [Patriot games Thomas Leo Clancy]   810   
 6   [La torre oscura II la llegada de los tres Ste...   810   
 7                       [The cleft Doris May Lessing]   820   
 8   [Un espía perfecto John Le Carré ; traducción ...   820   
 9   [Digital communications fundamentals and appli...   620   
 10  [El quinto evangelio Philipp Vandenberg ; trad...   830   
 
                          Tematicas   Llave  
 0       [[novela, estadounidense]]  906848  
 1       [[novela, estadounidense]]  824586  
 2   [[literatura, estadounidense]]  860720  
 3       [[cuento, estadounidense]]  551719  


In [ ]:
# Exportando el modelo
# sar_predictor_temas.export_model()

In [ ]:
now_time = time.strftime("%m%d%H%m")
filepath = "./sar_predictor_"+"+now_time"+".pkl"
with gzip.open(filepath, "wb") as f:
    pickled = pickle.dumps(sar_predictor, protocol=-1)
    optimized_pickle = pickletools.optimize(pickled)
    f.write(optimized_pickle)

# Content based KNN Clusters Temáticas

In [17]:
from content_knn_recommender import ContentKNNRecommender

In [18]:
content_predictor_temas = ContentKNNRecommender(df_pesos, n=50000, scale=5)

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.


In [19]:
content_predictor_temas.fit_evaluate()

Evaluando el algoritmo  ContentKNN ...
Calculando Precisión...
Pre fit base
Post fit base
Calculando matriz de similaridad basada en contenido...
...done.
Análisis Completado


Algorithm  RMSE       MAE       
ContentKNN 0.3704     0.1665    

Resumen:

RMSE:      Root Mean Squared Error. Valores más bajos significa mejor precisión.
MAE:       Mean Absolute Error. Valores más bajos significa mejor precisión.


In [ ]:
# Prediciendo para un usuario 
# cluster_eval = 92
# preds_temas_cb = content_predictor_temas.predict_for_cluster(cluster=cluster_eval)

In [ ]:
# Info cluster
# generateClusterInfo(cluster_eval)

In [ ]:
# Info Recomendaciones
# generateRecsInfopreds(preds_temas_cb)

In [ ]:
# content_predictor_temas.evaluator.algorithms[0].algorithm

In [168]:
import gzip, pickle, pickletools
import time
now_time = time.strftime("%m%d%H%m")
filepath = "./Models/content_predictor_temas_"+str(150)+"k_"+now_time+".pkl"
with gzip.open(filepath, "wb") as f:
    # pickled = pickle.dumps(content_predictor_temas.evaluator.algorithms[0].algorithm, protocol=4)
    pickled = pickle.dumps(content_predictor_temas, protocol=4)
    optimized_pickle = pickletools.optimize(pickled)
    f.write(optimized_pickle)

FileNotFoundError: [Errno 2] No such file or directory: './Models/content_predictor_temas_150k_10192210.pkl'

# RBM Temáticas

In [ ]:
from rbm_recommender import RBMRecommender

In [ ]:
rbm_predictor_temas = RBMRecommender(df_pesos.copy())

In [ ]:
rbm_predictor_temas.fit()

In [ ]:
#rbm_predictor_temas.predict()

In [ ]:
# Prediciendo para un usuario 
cluster_eval = 92
preds_temas_rbm = rbm_predictor_temas.predict_for_cluster(cluster=cluster_eval)

In [ ]:
# Info cluster
df_info_clus, df_clus_deweys, df_clus_tems = generateClusterInfo(cluster_eval)

In [ ]:
# Info Recomendaciones
df_info_pred_rbm_temas, df_pred_rbm_temas_deweys, df_pred_rbm_temas_tems = generateRecsInfopreds(preds_temas_rbm)

In [ ]:
intersection = df_clus_tems & df_pred_rbm_temas_tems
intersection

In [ ]:
difference = df_pred_rbm_temas_tems - df_clus_tems
difference

In [ ]:
# rbm_predictor_temas.export_model()

In [ ]:
import dill

In [2]:
# import gzip, pickle, pickletools
# import time
# now_time = time.strftime("%m%d%H%m")
# filepath = basePath + "UAQUE/Models/rbm_predictor_temas"+str(100)+"k_"+now_time+".pkl"
# with gzip.open(filepath, "wb") as f:
#     # pickled = pickle.dumps(rbm_predictor_temas, protocol=4)
#     pickled = dill.dumps(rbm_predictor_temas, protocol=4)
#     # optimized_pickle = pickletools.optimize(pickled)
#     # f.write(optimized_pickle)

# SVD

In [17]:
from svd_recommender import SVDRecommender

System version: 3.7.11 (default, Jul 27 2021, 07:03:16) 
[Clang 10.0.0 ]
Surprise version: 1.1.1


In [18]:
svd_predictor = SVDRecommender(df_pesos.copy())

In [19]:
svd_predictor.fit()

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 20
Processing epoch 21
Processing epoch 22
Processing epoch 23
Processing epoch 24
Processing epoch 25
Processing epoch 26
Processing epoch 27
Processing epoch 28
Processing epoch 29
Took 8.964098457999995 seconds for training.


In [20]:
svd_predictor.predict()

Took 375.24603912500004 seconds for prediction.


In [21]:
# Prediciendo para un usuario 
cluster_eval = 110
preds_svd = svd_predictor.predict_for_cluster(cluster=cluster_eval)

In [ ]:
# df_info_clus, df_clus_deweys, df_clus_tems = generateClusterInfo(cluster_eval)

In [22]:
# Info Recomendaciones
df_info_pred_svd, df_pred_svd_deweys, df_pred_svd_tems = generateRecsInfopreds(preds_svd)

,Titulo,Dewey,Tematicas,Llave
0,[Infancia Adolescencia Juventud León N. Tolstoi],920,[[none]],922502
1,[Gabo memorias de una vida mágica guión Óscar ...,740,"[[novela, gráfico, colombia]]",913393
2,[Los modelos pedagógicos Julián de Zubiría Sam...,370,"[[metodología, pedagogía]]",352812
3,[The architect reconstructing her practice edi...,720,"[[mujer, arquitecta]]",732373
4,[La inteligencia de una máquina una filosofía ...,790,[[cine]],899023
5,[Symposium on J. L. Austin edited by K. T. Fann],190,"[[filosofía, inglés]]",867805
6,[Alimentación y nutrición manual teórico-práct...,610,"[[nutrición, manual]]",873478
7,[Guía para la identificación y el cultivo de a...,580,"[[orquídea, cultivo, cundinamarca, colombia]]",921502
8,[Tendencias actuales de la profesión contable ...,650,"[[contabilidad, financiero], [contabilidad, fi...",345639
9,[Como ayudar a los preescolares pasivos y agre...,370,[],287639


['920' '740' '370' '720' '790' '190' '610' '580' '650']
['920' '740' '370' '720' '790' '190' '610' '580' '650']


In [4]:
# intersection = df_clus_tems & df_pred_svd_tems
# intersection

In [5]:
# difference = df_pred_svd_tems - 
# difference

In [6]:
#svd_predictor.export_model()

In [23]:
import gzip, pickle, pickletools
import time
now_time = time.strftime("%m%d%H%m")
filepath = "./svd_predictor"+str(100)+"k_"+now_time+".pkl"
with gzip.open(filepath, "wb") as f:
    pickled = pickle.dumps(svd_predictor, protocol=4)
    optimized_pickle = pickletools.optimize(pickled)
    f.write(optimized_pickle)

# Vowpal Wabbit

In [ ]:
%load_ext autoreload
%autoreload 2
from vowpal_recommender import VowpalRecommender

In [ ]:
data_pred = df_pesos.copy()

In [ ]:
vowpal_predictor = VowpalRecommender(data_pred)

In [ ]:
vowpal_predictor.fit()

In [ ]:
vowpal_predictor.predict()

In [ ]:
# Prediciendo para un usuario 
cluster_eval = 92
preds_vowpal = vowpal_predictor.predict_for_cluster(cluster=cluster_eval)

In [ ]:
df_info_clus, df_clus_deweys, df_clus_tems = generateClusterInfo(cluster_eval)

In [ ]:
# Info Recomendaciones
df_info_pred_vowpal, df_pred_vowpal_deweys, df_pred_vowpal_tems = generateRecsInfopreds(preds_vowpal)

In [ ]:
intersection = df_clus_tems & df_pred_vowpal_tems
intersection

In [ ]:
difference = df_pred_vowpal_tems - df_clus_tems
difference

In [ ]:
import gzip, pickle, pickletools
import time
now_time = time.strftime("%m%d%H%m")
filepath = basePath + "UAQUE/Models/vowpal_predictor"+str(100)+"k_"+now_time+".pkl"
with gzip.open(filepath, "wb") as f:
    pickled = pickle.dumps(svd_predictor, protocol=4)
    optimized_pickle = pickletools.optimize(pickled)
    f.write(optimized_pickle)